# Prepare Environment

In [ ]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import plotly
import plotly.express as px
import numpy as np
from statsmodels.tsa.api import VAR
import urllib.request

plotly.offline.init_notebook_mode()

NOW = datetime.now()
TODAY = NOW.date()

print('Aktualisiert:', NOW)

# Initialize Data

In [ ]:
STATE_NAMES = ['Burgenland', 'Kärnten', 'Niederösterreich', 
               'Oberösterreich', 'Salzburg', 'Steiermark', 
               'Tirol', 'Vorarlberg', 'Wien']

# TODO: Genauer recherchieren!
EVENTS = {'1. Lockdown': (np.datetime64('2020-03-20'), np.datetime64('2020-04-14'), 
                          'red', 'inside top left'),
          '1. Maskenpflicht': (np.datetime64('2020-03-30'), np.datetime64('2020-06-15'), 
                               'yellow', 'inside bottom left'),
          '2. Maskenpflicht': (np.datetime64('2020-07-24'), np.datetime64(TODAY), 
                               'yellow', 'inside bottom left'),
          'Soft Lockdown': (np.datetime64('2020-11-03'), np.datetime64('2020-11-17'), 
                            'orange', 'inside top left'),
          '2. Lockdown': (np.datetime64('2020-11-17'), np.datetime64(TODAY), 
                          'red', 'inside top left')}

def load_data(URL, date_columns):
    data_file = Path(URL).name
    
    try:
        # Only download the data if we don't have it, to avoid
        # excessive server access during local development
        with open(data_file):
            print("Using local", data_file)
    except FileNotFoundError:    
        print("Downloading", URL)
        urllib.request.urlretrieve(URL, data_file)

    return pd.read_csv(data_file, sep=';', parse_dates=date_columns, infer_datetime_format=True, dayfirst=True)

raw_data = load_data("https://covid19-dashboard.ages.at/data/CovidFaelle_Timeline.csv", [0])
additional_data = load_data("https://covid19-dashboard.ages.at/data/CovidFallzahlen.csv", [0, 2])

In [ ]:
cases = raw_data.query("Bundesland == 'Österreich'")
cases.insert(0, 'AnzahlFaelle_avg7', cases.AnzahlFaelle7Tage / 7)
time = cases.Time
cases

In [ ]:
tests = additional_data.query("Bundesland == 'Alle'")
tests.insert(2, 'TagesTests', np.concatenate([[np.nan], np.diff(tests.TestGesamt)]))
tests.insert(3, 'TagesTests_avg7', np.concatenate([[np.nan] * 7, (tests.TestGesamt.values[7:] - tests.TestGesamt.values[:-7])/7]))
tests.insert(0, 'Time', tests.MeldeDatum)
tests

# Fallzahlen Österreich (gesamt)

In [ ]:
fig = px.line(cases, x='Time', y=["AnzahlFaelle", "AnzahlFaelle_avg7"], log_y=True, title="Fallzahlen")
fig.add_scatter(x=tests.Time, y=tests.TagesTests, name='Tests')
for name, (begin, end, color, pos) in EVENTS.items():
    fig.add_vrect(x0=begin, x1=end, name=name, fillcolor=color, opacity=0.2, 
                  annotation={'text': name}, annotation_position=pos)
fig.show()

# Fälle Pro Test

In [ ]:
all_data = tests.merge(cases, on='Time', how='outer')
all_data.insert(1, 'PosRate', all_data.AnzahlFaelle / all_data.TagesTests)
all_data.insert(1, 'PosRate_avg7', all_data.AnzahlFaelle_avg7 / all_data.TagesTests_avg7)

fig = px.line(all_data, x='Time', y=['PosRate', 'PosRate_avg7'], log_y=False, title="Anteil Positiver Tests")
for name, (begin, end, color, pos) in EVENTS.items():
    fig.add_vrect(x0=begin, x1=end, name=name, fillcolor=color, opacity=0.2, 
                  annotation={'text': name}, annotation_position=pos)
fig.show()

# Wachstumsraten Bundesländer

In [ ]:
states = []
rates = []
for state_name, state_data in raw_data.groupby('Bundesland'):
    x = np.log2(state_data.AnzahlFaelle7Tage)
    rate = 2**np.array(np.diff(x))
    
    rates.append(rate)
    states.append(state_name)
growth = pd.DataFrame({n: r for n, r in zip(states, rates)})

fig = px.line(growth, x=time[1:], y=STATE_NAMES, title='Wachstumsrate')
fig.update_layout(yaxis=dict(range=[0.25, 4]))
fig.show()

In [ ]:
model = VAR(growth[150:][STATE_NAMES])
res = model.fit(1)
res.summary()